# XGBoost

In [1]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
import pandas as pd
import numpy as np

In [2]:
train_XY_url = 'https://minio-api.all.ar/orga-datos/properati_argentina_2021_train_XY_preprocesado.csv'
df_train_so = pd.read_csv(train_XY_url)

In [3]:
test_XY_url = 'https://minio-api.all.ar/orga-datos/properati_argentina_2021_test_XY_preprocesado.csv'
df_test_XY = pd.read_csv(test_XY_url)

In [4]:
train_outlier_XY_url = 'https://minio-api.all.ar/orga-datos/properati_argentina_2021_train_XY_preprocesado_y_outlier.csv'
df_train_XY = pd.read_csv(train_outlier_XY_url)

In [5]:
df_train_XY = df_train_XY.drop(['año_publicacion', 'property_coverage_percent'], axis=1)

In [6]:
df_validation_XY = df_train_XY.sample(frac=0.2)
df_validation_XY_copy = df_validation_XY.copy()
df_train_XY_copy = df_train_XY.copy()
df_test_XY_copy = df_test_XY.copy()

In [7]:
RANDOM_STATE = 3

## Preprocesamiento plus

In [8]:
from sklearn.preprocessing import OneHotEncoder

enc_language = OneHotEncoder()
X_t = df_train_XY[['property_type']]
X_t = enc_language.fit_transform(X_t).toarray()
language_columns = enc_language.get_feature_names_out()
#agrego las columnas al df_train
df_train_XY[language_columns] = X_t

df_train_XY = df_train_XY.drop(['property_type'], axis=1)

#repito el mismo proceso para df_validation
enc_language_val = OneHotEncoder()
X_v = df_validation_XY.loc[:,['property_type']]
X_v = enc_language_val.fit_transform(X_v).toarray()
language_columns_val = enc_language_val.get_feature_names_out()
df_validation_XY[language_columns_val] = X_v
df_validation_XY = df_validation_XY.drop(['property_type'], axis=1)

#repito el mismo proceso para df_test_XY
enc_language_val = OneHotEncoder()
X_v = df_test_XY.loc[:,['property_type']]
X_v = enc_language_val.fit_transform(X_v).toarray()
language_columns_val = enc_language_val.get_feature_names_out()
df_test_XY[language_columns_val] = X_v
df_test_XY = df_test_XY.drop(['property_type'], axis=1)


In [9]:
df_train_XY = df_train_XY.drop(['property_title'], axis=1)
df_validation_XY = df_validation_XY.drop(['property_title'], axis=1)
df_test_XY = df_test_XY.drop(['property_title'], axis=1)

In [10]:
df_train_XY = df_train_XY.drop(['place_l3'], axis=1)
df_validation_XY = df_validation_XY.drop(['place_l3'], axis=1)
df_test_XY = df_test_XY.drop(['place_l3'], axis=1)

In [11]:
df_train_XY = df_train_XY.drop(['start_date', 'end_date'], axis=1)
df_validation_XY = df_validation_XY.drop(['start_date', 'end_date'], axis=1)
df_test_XY = df_test_XY.drop(['start_date', 'end_date'], axis=1)

In [12]:
df_validation_Y = df_validation_XY['property_price']
df_validation_X = df_validation_XY.drop(['property_price'], axis=1)
df_train_Y = df_train_XY['property_price']
df_train_X = df_train_XY.drop(['property_price'], axis=1)
df_test_Y = df_test_XY['property_price']
df_test_X = df_test_XY.drop(['property_price'], axis=1)

## Entrenamiento Básico

In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score

In [14]:
# Crear el regresor XGBRFRegressor
model_uno = xgb.XGBRFRegressor()

# Entrenar el modelo
model_uno.fit(df_train_X, df_train_Y)

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=None, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=None, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=None, reg_alpha=None, ...)

In [15]:
# Realizar predicciones en el conjunto de prueba
pred_y_uno = model_uno.predict(df_validation_X)


In [16]:
# Calcular el error cuadrático medio (MSE)
mse_uno = mean_squared_error(df_validation_Y, pred_y_uno)
print("Error cuadrático medio (MSE):", mse_uno)

Error cuadrático medio (MSE): 5831136967.90697


## Buscamos Hiper parámetros

In [17]:
# Definir los hiperparámetros a buscar
param_grid = {
'n_estimators': [30,40,80],
'max_depth': [9, 12, 20],
'learning_rate': [0.01, 0.1, 0.5],
'subsample': [0.8, 0.9, 1.0],
'colsample_bynode': [0.8, 0.9, 1.0],
'reg_alpha': [0, 0.1, 0.5],
'reg_lambda': [0, 0.1, 0.5],
'min_child_weight': [1, 3, 5],
'gamma': [0, 0.1, 0.5],
'objective': ['reg:squarederror', 'reg:squaredlogerror']
}

In [18]:
# Realizar la búsqueda aleatoria de hiperparámetros
random_search = RandomizedSearchCV(model_uno, param_distributions=param_grid, n_iter=20, cv=4, random_state=RANDOM_STATE, verbose=3)

In [19]:
random_search.fit(df_train_X, df_train_Y)

Fitting 4 folds for each of 20 candidates, totalling 80 fits
[CV 1/4] END colsample_bynode=0.8, gamma=0.1, learning_rate=0.1, max_depth=9, min_child_weight=3, n_estimators=30, objective=reg:squarederror, reg_alpha=0, reg_lambda=0, subsample=0.8;, score=0.166 total time=   0.4s
[CV 2/4] END colsample_bynode=0.8, gamma=0.1, learning_rate=0.1, max_depth=9, min_child_weight=3, n_estimators=30, objective=reg:squarederror, reg_alpha=0, reg_lambda=0, subsample=0.8;, score=0.169 total time=   0.4s
[CV 3/4] END colsample_bynode=0.8, gamma=0.1, learning_rate=0.1, max_depth=9, min_child_weight=3, n_estimators=30, objective=reg:squarederror, reg_alpha=0, reg_lambda=0, subsample=0.8;, score=0.164 total time=   0.4s
[CV 4/4] END colsample_bynode=0.8, gamma=0.1, learning_rate=0.1, max_depth=9, min_child_weight=3, n_estimators=30, objective=reg:squarederror, reg_alpha=0, reg_lambda=0, subsample=0.8;, score=0.162 total time=   0.4s
[CV 1/4] END colsample_bynode=0.8, gamma=0, learning_rate=0.1, max_dept

RandomizedSearchCV(cv=4,
                   estimator=XGBRFRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            max_bin=None,
                                            max_cat_threshold=Non...
                                            random_state=None, reg_alpha=None, ...),
                   n_iter=20,
                   param_distributions={'colsample_bynode': [0.8, 0.9, 1.0],
                                        'gamma': [0, 0.1, 0.5],
                                        'learning_rate': [0.01, 0.1, 0.5],
                                        'max_depth': [9, 12, 20],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [30, 40, 80],
                                        'objective': ['reg:squarederror',
                                                      'reg:squaredlogerror'],
                                        'reg_alpha': [0, 0.1, 0.5],
                                        'reg_lambda': [0, 0.1, 0.5],
                                        'subsample': [0.8, 0.9, 1.0]},
                   random_state=3, verbose=3)

In [20]:
# Obtener los mejores hiperparámetros y el mejor modelo
best_params = random_search.best_params_
best_model = random_search.best_estimator_

In [21]:
random_search.best_estimator_

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=1.0,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=0.5, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.5, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=20, max_leaves=None,
               min_child_weight=3, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=80, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror', ...)

In [22]:
random_search.best_params_

{'subsample': 0.8,
 'reg_lambda': 0,
 'reg_alpha': 0.5,
 'objective': 'reg:squarederror',
 'n_estimators': 80,
 'min_child_weight': 3,
 'max_depth': 20,
 'learning_rate': 0.5,
 'gamma': 0.5,
 'colsample_bynode': 1.0}

In [23]:
# Realizar predicciones en el conjunto de prueba con el mejor modelo
pred_y_dos = best_model.predict(df_validation_X)


In [24]:
# Calcular el error cuadrático medio (MSE)
mse_dos = mean_squared_error(df_validation_Y, pred_y_dos)

# Calcular el error absoluto medio (MAE)
mae_dos = mean_absolute_error(df_validation_Y, pred_y_dos)

# Calcular la raíz del error cuadrático medio (RMSE)
rmse_dos = np.sqrt(mse_dos)

print("Error cuadrático medio (MSE):", mse_dos)
print("Error absoluto medio (MAE):", mae_dos)
print("Raíz del error cuadrático medio (RMSE):", rmse_dos)
print("Mejores hiperparámetros:", best_params)

Error cuadrático medio (MSE): 11144455273.136312
Error absoluto medio (MAE): 62533.27721657511
Raíz del error cuadrático medio (RMSE): 105567.30210219599
Mejores hiperparámetros: {'subsample': 0.8, 'reg_lambda': 0, 'reg_alpha': 0.5, 'objective': 'reg:squarederror', 'n_estimators': 80, 'min_child_weight': 3, 'max_depth': 20, 'learning_rate': 0.5, 'gamma': 0.5, 'colsample_bynode': 1.0}


## Modelo supre básico para comparar

In [25]:
from sklearn.linear_model import LinearRegression
model_tres = LinearRegression()

# Entrenar el modelo
model_tres.fit(df_train_X, df_train_Y)

# Realizar predicciones en el conjunto de prueba
pred_tres = model_tres.predict(df_validation_X)

# Calcular el error cuadrático medio (MSE)
mse_tres = mean_squared_error(df_validation_Y, pred_tres)
print("Error cuadrático medio (MSE) - Regresión Lineal:", mse_tres)

Error cuadrático medio (MSE) - Regresión Lineal: 10733351366.970558


## Con Normalización

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
# Normalizar los datos de entrada
scaler = StandardScaler()
df_train_X_normalizado = scaler.fit_transform(df_train_X)
df_validation_X_normalizado = scaler.transform(df_validation_X)
df_test_X_normalizado = scaler.transform(df_test_X)

In [28]:
# Definir los hiperparámetros a buscar
param_grid = {
    'n_estimators': [10, 20, 30],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bynode': [0.8, 0.9, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.5],
    'objective': ['reg:squarederror', 'reg:squaredlogerror']
}

In [29]:
# Crear el regresor XGBRFRegressor
model_cuatro = xgb.XGBRFRegressor()

In [30]:
# Realizar la búsqueda aleatoria de hiperparámetros
random_search = RandomizedSearchCV(model_cuatro, param_distributions=param_grid, n_iter=8, cv=3, random_state=RANDOM_STATE, verbose=1)
random_search.fit(df_train_X_normalizado, df_train_Y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRFRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            max_bin=None,
                                            max_cat_threshold=Non...
                                            random_state=None, reg_alpha=None, ...),
                   n_iter=8,
                   param_distributions={'colsample_bynode': [0.8, 0.9, 1.0],
                                        'gamma': [0, 0.1, 0.5],
                                        'learning_rate': [0.01, 0.1, 0.5],
                                        'max_depth': [3, 6, 9],
                                        'min_child_weight': [1, 3, 5],
                                        'n_estimators': [10, 20, 30],
                                        'objective': ['reg:squarederror',
                                                      'reg:squaredlogerror'],
                                        'reg_alpha': [0, 0.1, 0.5],
                                        'reg_lambda': [0, 0.1, 0.5],
                                        'subsample': [0.8, 0.9, 1.0]},
                   random_state=3, verbose=1)

In [31]:
# Obtener los mejores hiperparámetros y el mejor modelo
best_params = random_search.best_params_
best_model = random_search.best_estimator_

In [32]:
# Realizar predicciones en el conjunto de entrenamiento
pred_train_y_cuatro = best_model.predict(df_train_X_normalizado)
# Realizar predicciones en el conjunto de prueba con el mejor modelo
pred_val_y_cuatro = best_model.predict(df_validation_X_normalizado)

In [33]:
# Calcular el error cuadrático medio (MSE) en los datos de entrenamiento
mse_train_cuatro = mean_squared_error(df_train_Y, pred_train_y_cuatro)

# Calcular el error absoluto medio (MAE) en los datos de entrenamiento
mae_train_cuatro = mean_absolute_error(df_train_Y, pred_train_y_cuatro)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de entrenamiento
rmse_train_cuatro = np.sqrt(mae_train_cuatro)

# Calcular el error cuadrático medio (MSE) en los datos de prueba
mse_val_cuatro = mean_squared_error(df_validation_Y, pred_val_y_cuatro)

# Calcular el error absoluto medio (MAE) en los datos de prueba
mae_val_cuatro = mean_absolute_error(df_validation_Y, pred_val_y_cuatro)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de prueba
rmse_val_cuatro = np.sqrt(mse_val_cuatro)

# Comparar los errores de entrenamiento y prueba
print("Error cuadrático medio (MSE) - Entrenamiento:", mse_train_cuatro)
print("Error cuadrático medio (MSE) - Prueba:", mse_val_cuatro)
print("Error absoluto medio (MAE) - Entrenamiento:", mae_train_cuatro)
print("Error absoluto medio (MAE) - Prueba:", mae_val_cuatro)
print("Raíz del error cuadrático medio (RMSE) - Entrenamiento:", rmse_train_cuatro)
print("Raíz del error cuadrático medio (RMSE) - Prueba:", rmse_val_cuatro)

Error cuadrático medio (MSE) - Entrenamiento: 13329324054.76223
Error cuadrático medio (MSE) - Prueba: 13312467421.004446
Error absoluto medio (MAE) - Entrenamiento: 68099.91310644851
Error absoluto medio (MAE) - Prueba: 68039.0033478862
Raíz del error cuadrático medio (RMSE) - Entrenamiento: 260.9596005255383
Raíz del error cuadrático medio (RMSE) - Prueba: 115379.6664105268


## Ahora usamos GridSearchCV

In [34]:
df_validation_X_normalizado_copy = df_validation_X_normalizado.copy()
df_train_X_normalizado_copy = df_train_X_normalizado.copy()
df_test_X_normalizado_copy = df_test_X_normalizado.copy()

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Cantidad de combinaciones que quieres probar
n = 3

# Conjunto de parámetros que quieres usar
param_grid = {
'n_estimators': [30,40],
'max_depth': [9, 12, 20],
'learning_rate': [0.1, 0.5],
'subsample': [0.9, 1.0],
'colsample_bynode': [0.9, 1.0],
'reg_alpha': [0.1, 0.5],
'reg_lambda': [0.1, 0.5],
'min_child_weight': [1, 3, 5],
'gamma': [0.1, 0.5],
'objective': ['reg:squarederror', 'reg:squaredlogerror']
}

# Cantidad de splits para el Cross Validation
folds = 2

# Kfold estratificado
kfoldcv = StratifiedKFold(n_splits=folds)

# Modelo
model_cinco = xgb.XGBRFRegressor()

# Métrica
from sklearn.metrics import make_scorer, mean_absolute_error
scorer_fn = make_scorer(mean_absolute_error, greater_is_better=False)

# GridSearch CV
gridcv = GridSearchCV(estimator=model_cinco, param_grid=param_grid, scoring=scorer_fn, cv=kfoldcv, return_train_score=True, verbose=10)

# Mostrar los resultados de la búsqueda
gridcv.fit(df_train_X_normalizado, df_train_Y)

gridcv.cv_results_

Fitting 2 folds for each of 2304 candidates, totalling 4608 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


Streaming output truncated to the last 5000 lines.
[CV 1/2; 1055/2304] START colsample_bynode=0.9, gamma=0.5, learning_rate=0.5, max_depth=12, min_child_weight=5, n_estimators=40, objective=reg:squaredlogerror, reg_alpha=0.5, reg_lambda=0.5, subsample=0.9
[CV 1/2; 1055/2304] END colsample_bynode=0.9, gamma=0.5, learning_rate=0.5, max_depth=12, min_child_weight=5, n_estimators=40, objective=reg:squaredlogerror, reg_alpha=0.5, reg_lambda=0.5, subsample=0.9;, score=(train=-203274.922, test=-202981.842) total time=   0.1s
[CV 2/2; 1055/2304] START colsample_bynode=0.9, gamma=0.5, learning_rate=0.5, max_depth=12, min_child_weight=5, n_estimators=40, objective=reg:squaredlogerror, reg_alpha=0.5, reg_lambda=0.5, subsample=0.9
[CV 2/2; 1055/2304] END colsample_bynode=0.9, gamma=0.5, learning_rate=0.5, max_depth=12, min_child_weight=5, n_estimators=40, objective=reg:squaredlogerror, reg_alpha=0.5, reg_lambda=0.5, subsample=0.9;, score=(train=-202981.842, test=-203274.922) total time=   0.1s
[CV

{'mean_fit_time': array([0.59511077, 0.2952795 , 0.28171825, ..., 0.09409881, 0.08078575,
        0.10082114]),
 'std_fit_time': array([0.30130637, 0.0134151 , 0.00309372, ..., 0.00268102, 0.00172186,
        0.01065862]),
 'mean_score_time': array([0.03700161, 0.03026223, 0.0312351 , ..., 0.00907123, 0.00575078,
        0.00761628]),
 'std_score_time': array([6.56652451e-03, 1.12056732e-04, 2.47836113e-04, ...,
        1.57034397e-03, 8.34465027e-07, 7.82012939e-05]),
 'param_colsample_bynode': masked_array(data=[0.9, 0.9, 0.9, ..., 1.0, 1.0, 1.0],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=[0.1, 0.1, 0.1, ..., 0.5, 0.5, 0.5],
              mask=[False, False, False, ..., False, False, False],
        fill_value='?',
             dtype=object),
 'param_learning_rate': masked_array(data=[0.1, 0.1, 0.1, ..., 0.5, 0.5, 0.5],
              mask=[False, False, False, ..., False, F

In [37]:
best_model = gridcv.best_estimator_

In [38]:
# Realizar predicciones en el conjunto de entrenamiento
pred_train_y_cinco = best_model.predict(df_train_X_normalizado)
# Realizar predicciones en el conjunto de prueba con el mejor modelo
pred_val_y_cinco = best_model.predict(df_validation_X_normalizado)

In [39]:
# Calcular el error cuadrático medio (MSE) en los datos de entrenamiento
mse_train_cinco = mean_squared_error(df_train_Y, pred_train_y_cinco)

# Calcular el error absoluto medio (MAE) en los datos de entrenamiento
mae_train_cinco = mean_absolute_error(df_train_Y, pred_train_y_cinco)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de entrenamiento
rmse_train_cinco = np.sqrt(mse_train_cinco)

# Calcular el error cuadrático medio (MSE) en los datos de prueba
mse_val_cinco = mean_squared_error(df_validation_Y, pred_val_y_cinco)

# Calcular el error absoluto medio (MAE) en los datos de prueba
mae_val_cinco = mean_absolute_error(df_validation_Y, pred_val_y_cinco)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de prueba
rmse_val_cinco = np.sqrt(mse_val_cinco)

# Comparar los errores de entrenamiento y prueba
print("Error cuadrático medio (MSE) - Entrenamiento:", mse_train_cinco)
print("Error cuadrático medio (MSE) - Prueba:", mse_val_cinco)
print("Error absoluto medio (MAE) - Entrenamiento:", mae_train_cinco)
print("Error absoluto medio (MAE) - Prueba:", mae_val_cinco)
print("Raíz del error cuadrático medio (RMSE) - Entrenamiento:", rmse_train_cinco)
print("Raíz del error cuadrático medio (RMSE) - Prueba:", rmse_val_cinco)

Error cuadrático medio (MSE) - Entrenamiento: 11319822129.318222
Error cuadrático medio (MSE) - Prueba: 11403405875.356577
Error absoluto medio (MAE) - Entrenamiento: 62968.63964632755
Error absoluto medio (MAE) - Prueba: 63046.104818521715
Raíz del error cuadrático medio (RMSE) - Entrenamiento: 106394.65272897047
Raíz del error cuadrático medio (RMSE) - Prueba: 106786.73080189587


In [40]:
resultados = {
    "Modelo Uno": mse_uno,
    "Modelo Dos": mse_dos,
    "Modelo Cuatro - Prueba": mse_val_cuatro,
    "Modelo Cinco - Prueba": mse_val_cinco
}

# Encontrar el modelo con el MSE más bajo
mejor_modelo = min(resultados, key=resultados.get)
mejor_mse = resultados[mejor_modelo]

print("El mejor modelo es:", mejor_modelo)
print("Error cuadrático medio (RMSE) del mejor modelo:", np.sqrt(mejor_mse))

El mejor modelo es: Modelo Uno
Error cuadrático medio (RMSE) del mejor modelo: 76361.88164200101


In [41]:
# Resultados de los modelos
resultados_mae = {
    "Modelo Dos": mae_dos,
    "Modelo Cuatro - Prueba": mae_val_cuatro,
    "Modelo Cinco - Prueba": mae_val_cinco
}

# Encontrar el modelo con el MAE más bajo
mejor_modelo_mae = min(resultados_mae, key=resultados_mae.get)
mejor_mae = resultados_mae[mejor_modelo_mae]

print("El mejor modelo en términos de Error Absoluto Medio (MAE) es:", mejor_modelo_mae)
print("Error absoluto medio (MAE) del mejor modelo:", mejor_mae)

El mejor modelo en términos de Error Absoluto Medio (MAE) es: Modelo Dos
Error absoluto medio (MAE) del mejor modelo: 62533.27721657511


## Probamos en test el mejor modelo en validación

In [42]:
best_model = random_search.best_estimator_
pred_test_y = best_model.predict(df_test_X_normalizado)

In [43]:
# Calcular el error cuadrático medio (MSE) en los datos de entrenamiento
mse_test = mean_squared_error(df_test_Y, pred_test_y)

# Calcular el error absoluto medio (MAE) en los datos de entrenamiento
mae_test = mean_absolute_error(df_test_Y, pred_test_y)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de entrenamiento
rmse_test = np.sqrt(mse_test)

# Errores test
print("Error cuadrático medio (MSE) - Test:", mse_test)
print("Error absoluto medio (MAE) - Test:", mae_test)
print("Raíz del error cuadrático medio (RMSE) - Test:", rmse_test)

Error cuadrático medio (MSE) - Test: 76425807736.26505
Error absoluto medio (MAE) - Test: 87484.94506065312
Raíz del error cuadrático medio (RMSE) - Test: 276452.17983634176


In [44]:
import pickle

# Guardar el modelo entrenado en un archivo
with open('modelo_entrenado.pkl', 'wb') as file:
    pickle.dump(best_model, file)

In [45]:
# Cargar el modelo desde el archivo
with open('modelo_entrenado.pkl', 'rb') as file:
    loaded_model_pickle = pickle.load(file)

In [46]:
# Realizar predicciones en el conjunto de prueba con el modelo cargado con pickle
pred_test_y_loaded_pickle = loaded_model_pickle.predict(df_test_X_normalizado)

In [47]:
# Calcular el error cuadrático medio (MSE) en los datos de entrenamiento
mse_test_bis = mean_squared_error(df_test_Y, pred_test_y_loaded_pickle)

# Calcular el error absoluto medio (MAE) en los datos de entrenamiento
mae_test_bis = mean_absolute_error(df_test_Y, pred_test_y_loaded_pickle)

# Calcular la raíz del error cuadrático medio (RMSE) en los datos de entrenamiento
rmse_test_bis = np.sqrt(mse_test_bis)

# Errores test
print("Error cuadrático medio (MSE) - Test:", mse_test_bis)
print("Error absoluto medio (MAE) - Test:", mae_test_bis)
print("Raíz del error cuadrático medio (RMSE) - Test:", rmse_test_bis)

Error cuadrático medio (MSE) - Test: 76425807736.26505
Error absoluto medio (MAE) - Test: 87484.94506065312
Raíz del error cuadrático medio (RMSE) - Test: 276452.17983634176
